In [5]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train_dogs = x_train[y_train[:, 0] == 5]
y_train_dogs = y_train[y_train[:, 0] == 5]
x_test_dogs = x_test[y_test[:, 0] == 5]
y_test_dogs = y_test[y_test[:, 0] == 5]

with tf.device('/CPU:0'):
    x_train_dogs = tf.image.resize(x_train_dogs, (224, 224))
    x_test_dogs = tf.image.resize(x_test_dogs, (224, 224))

x_train_dogs = x_train_dogs / 255.0
x_test_dogs = x_test_dogs / 255.0

y_train_dogs = to_categorical(y_train_dogs, 10)
y_test_dogs = to_categorical(y_test_dogs, 10)

data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.1
)

data_generator.fit(x_train_dogs)
train_generator = data_generator.flow(x_train_dogs, y_train_dogs, batch_size=8)
batch_size = 8

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_generator, 
                    steps_per_epoch=len(x_train_dogs) // batch_size, 
                    epochs=10, 
                    validation_data=(x_test_dogs, y_test_dogs), 
                    verbose=1)

test_loss, test_acc = model.evaluate(x_test_dogs, y_test_dogs, verbose=2)
print(f'Test accuracy: {test_acc*100:.2f}%')

import numpy as np
from tensorflow.keras.preprocessing import image

def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255.0
    return img

def predict_dog(model, image_path):
    img = preprocess_image(image_path)
    prediction = model.predict(img)
    class_index = np.argmax(prediction)
    return class_index, prediction[0][class_index]

image_path = r"C:\Users\uday\Downloads\dog.jpg"
class_index, confidence = predict_dog(model, image_path)


Epoch 1/10
625/625 [==============================] - 250s 393ms/step - loss: 0.0039 - accuracy: 0.9986 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/10
625/625 [==============================] - 245s 392ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/10
625/625 [==============================] - 241s 385ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/10
625/625 [==============================] - 241s 385ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/10
625/625 [==============================] - 243s 389ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/10
625/625 [==============================] - 246s 394ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/10
625/625 [==============================] - 246s 394ms/step - loss: 0.

In [6]:
if class_index == 5:
    print("The model predicts that it's a dog!")
else:
    print("The model predicts that it's not a dog.")

The model predicts that it's a dog!
